In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *

In [2]:
spark=SparkSession.builder.appName('desafio-python').getOrCreate()

In [3]:
from IPython.core.display import HTML
HTML(r"""
<style>
    .output-plaintext, .output-stream, .output {
        white-space: pre !important;
    }
    pre { white-space: pre !important; }
</style>
""")

In [4]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.truncate', 30)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 20)

In [5]:
# dataframes_dict = {}
# dataframes_dict['metadata_de_facturas']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/bkpf_vbrk.parquet")
# dataframes_dict['facturas_por_cliente']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/vbpa_kna1.parquet")
# dataframes_dict['materiales']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/vbrp_mara_marm.parquet")
# dataframes_dict['paises']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/T005T.parquet")

########################################################################################################################################

# dataframes_dict = {}
# dataframes_dict['metadata_de_facturas']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/bkpf_vbrk.parquet")
# dataframes_dict['facturas_por_cliente']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/vbpa_kna1.parquet")
# dataframes_dict['materiales']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/vbrp_mara_marm.parquet")
# dataframes_dict['paises']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/T005T.parquet")

########################################################################################################################################

dataframes_dict = {}
dataframes_dict['metadata_de_facturas']=spark.read.parquet("/Users/diego/Desktop/Tarea Python/bkpf_vbrk.parquet")
dataframes_dict['facturas_por_cliente']=spark.read.parquet("/Users/diego/Desktop/Tarea Python/vbpa_kna1.parquet")
dataframes_dict['materiales']=spark.read.parquet("/Users/diego/Desktop/Tarea Python/vbrp_mara_marm.parquet")
dataframes_dict['paises']=spark.read.parquet("/Users/diego/Desktop/Tarea Python/T005T.parquet")

In [6]:
dataframes_dict['facturas_por_cliente'].printSchema()
dataframes_dict['metadata_de_facturas'].printSchema()
dataframes_dict['materiales'].printSchema()
dataframes_dict['paises'].printSchema()

root
 |-- Numero_de_Cliente: string (nullable = true)
 |-- Mandante: string (nullable = true)
 |-- Factura: string (nullable = true)
 |-- Posicion_de_Factura: integer (nullable = true)
 |-- Tipo_de_Cliente: string (nullable = true)
 |-- Direccion: string (nullable = true)
 |-- Punto_descarga: string (nullable = true)
 |-- Clave_pais: string (nullable = true)
 |-- Indicador_direccion: string (nullable = true)
 |-- Descr_cliente_socio: string (nullable = true)
 |-- Zona_transporte: string (nullable = true)
 |-- Numero_personal: string (nullable = true)
 |-- Cliente_Solicitante: string (nullable = true)
 |-- Cliente_Pagador: string (nullable = true)
 |-- Cliente_Destinatario_de_Mercancia: string (nullable = true)
 |-- Cliente_Destinatario_de_factura: string (nullable = true)
 |-- Representante_de_ventas: string (nullable = true)
 |-- Pais: string (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- Poblacion: string (nullable = true)
 |-- Codigo_postal: string (nullable = true)
 |

#### Filtros por Mandante = 400

In [7]:
df_metadata_facturas = dataframes_dict['metadata_de_facturas'].where(col('Mandante') == '400')

In [8]:
df_facturas_por_cliente = dataframes_dict['facturas_por_cliente'].where(col('Mandante') == '400')

In [9]:
df_materiales = dataframes_dict['materiales'].where(col('Mandante') == '400')

In [10]:
df_paises = dataframes_dict['paises'].where(col('Mandante') == '400').where(col('Idioma') == 'S')
# Ocupar sólo "S" Y info mandante 400

#### SELECT DE TABLAS

In [11]:
df_metadata_facturas = df_metadata_facturas.select('Factura', 'Clase_factura', 'Ejercicio', 'Per_contable').where(col('Factura').isNotNull())


In [12]:
df_facturas_por_cliente = df_facturas_por_cliente.select('Factura','Posicion_de_Factura','Pais', 'Nombre', 'Numero_de_Cliente', 'Cliente_Solicitante', 'Cliente_Destinatario_de_Mercancia')\
                                                 .withColumnRenamed('Pais', 'Pais_cliente')

In [13]:
df_materiales = df_materiales.groupBy('Factura', 'Posicion', 'Material', 'Unidad_de_medida_Volumen_Destino', 'Pais').agg(sum(col('Volumen')).alias('Volumen'))\
                             .withColumnRenamed('Pais', 'Pais_factura')

In [14]:
df_paises = df_paises.select('Idioma', 'Pais' , 'Pais_descr')

#### Join tablas

In [19]:
def facturas_clientes_x_metadata_facturas(dataframes_dict):
    df_result = df_facturas_por_cliente.join(df_metadata_facturas, 'Factura', 'left')

    return {'add_facturas_clientes_x_metadata_facturas': df_result}

def materiales_x_clase_factura(dataframes_dict):
    df_metadata_facturas_clientes = dataframes_dict['add_facturas_clientes_x_metadata_facturas']
    df_facturas_por_cliente_2 = df_facturas_por_cliente.select('Factura', 'Cliente_Solicitante') 
    df_result = df_materiales.join(df_facturas_por_cliente_2, 'Factura', 'left')

    
    # join_cond = [df_materiales.Factura == df_metadata_facturas_clientes.Factura]
    # df_result = df_materiales.join(df_metadata_facturas_clientes, join_cond, 'left')\
    #                          .drop(df_metadata_facturas_clientes.Posicion_de_Factura)\
    #                          .drop(df_metadata_facturas_clientes.Factura)

    return {'add_materiales_x_clase_factura': df_result}

# def materiales_x_clase_factura_2(dataframes_dict):
#     df_metadata_facturas_clientes = dataframes_dict['add_facturas_clientes_x_metadata_facturas']
#     join_cond = [df_materiales.Factura == df_metadata_facturas_clientes.Factura, df_materiales.Posicion == df_metadata_facturas_clientes.Posicion_de_Factura]
#     df_result = df_materiales.join(df_metadata_facturas_clientes, join_cond, 'left')\
#                              .drop(df_metadata_facturas_clientes.Posicion_de_Factura)\
#                              .drop(df_metadata_facturas_clientes.Factura)

 #   return {'add_materiales_x_clase_factura': df_result}

# def paises_descripcion(dataframes_dict):
#     df_materiales_facturas = dataframes_dict['add_materiales_x_clase_factura']
#     df_result = df_materiales_facturas.join(df_paises,df_paises.Pais == df_materiales_facturas.Pais_cliente, 'left')\
#                                       .drop('Pais')

#     return {'add_paises_descripcion': df_result}

In [23]:
dataframes_dict['add_materiales_x_clase_factura'].where(col('Cliente_Solicitante').isNotNull())

Factura,Posicion,Material,Unidad_de_medida_Volumen_Destino,Pais_factura,Volumen,Cliente_Solicitante
0150113050,76,M100PLY000122,M3,CL,2.8934280639431615,17309
0150140755,23,M100PLY000107,M3,CL,2.8577280027594236,242801
0150140755,24,M100PLY000107,M3,CL,2.8577280027594236,242801
0933004035,10,M100PLY000248-002,M3,CL,4.465199987588047E-5,627
0150183724,16,M100PLY000037,M3,CL,2.8224,67622
0150141959,63,M460PPI000023,M3,CL,2.0646424568771176,C6108
0150141959,49,M460PPI000027,M3,CL,2.907359941413402,C6108
0150141959,128,M520MPI000035,M3,CL,3.7500632089942183,C6108
0150193854,31,M400PPF000156,M3,CL,2.1553912371702904,C6105
0150141345,12,M100PLY000188,M3,CL,2.7648,67622


In [22]:
dataframes_dict['add_facturas_clientes_x_metadata_facturas'].where(col('Cliente_Solicitante').isNotNull()).count()

2798

In [17]:
def main(dataframes_dict):

    functions_list = [
        facturas_clientes_x_metadata_facturas,
        materiales_x_clase_factura,
        paises_descripcion,
        # Otras transformaciones
    ]

    for function in functions_list:
        print(function)
        result_element = function(dataframes_dict)
        dataframes_dict.update(result_element)

    return dataframes_dict['add_paises_descripcion'] # Modificar al último dataframe retornado

#### Dataframe Resultado

In [18]:
dataframe_final = main(dataframes_dict).select('Cliente_Solicitante', 'Cliente_Destinatario_de_Mercancia', 'Factura', 'Clase_factura', 'Posicion', 'Material', 'Volumen', 
                                        'Unidad_de_medida_Volumen_Destino','Per_contable', 'Ejercicio', 'Numero_de_cliente', 'Nombre', 'Pais_cliente', 'Pais_descr')\
                                       .withColumnRenamed('Nombre', 'Nombre_cliente') 


<function facturas_clientes_x_metadata_facturas at 0x000001BF75AE7910>
<function materiales_x_clase_factura at 0x000001BF75AE7AC0>
<function paises_descripcion at 0x000001BF75AE69E0>


AttributeError: 'DataFrame' object has no attribute 'Pais_cliente'

#### Consultas

#### Cantidad de facturas por tipo de cliente 

In [ ]:
dataframe_final.where(col('Cliente_Destinatario_de_Mercancia').isNotNull()).count()

2800